In [1]:
#导包
from netCDF4 import Dataset
from tensorflow.keras import layers, models
from tensorflow.keras.layers import ConvLSTM2D,LSTM, BatchNormalization, LayerNormalization,Input, Conv3D, TimeDistributed, Flatten
from tensorflow.keras.layers import Concatenate,Conv2D,TimeDistributed, MaxPooling2D, Input, MaxPooling3D
from tensorflow.keras.layers import  Reshape,multiply
from tensorflow.keras.layers import Layer,Lambda,Dot,ReLU, Dense, Dropout, Activation, Flatten,Attention
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Sequential,Model,load_model
from tensorflow.keras.optimizers import Adam,SGD
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from tensorflow.keras.losses import MeanSquaredError
from sklearn.metrics import r2_score,accuracy_score,precision_score
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras import regularizers
import tensorflow as tf
from datetime import datetime
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from tensorflow.keras import models, layers
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
import tensorflow.keras as keras
from tensorflow.keras import layers, Sequential, regularizers
from tensorflow.keras.utils import  plot_model
# calculate RMSE
from sklearn.metrics import mean_squared_error
from math import sqrt
import pandas as pd 
import gc 
import gzip
import matplotlib.pyplot as plt
import numpy as np
import netCDF4 as nc
import match
import xarray as xr
import os 
%matplotlib inline

In [2]:
# 检查可用GPU数量
len(tf.config.experimental.list_physical_devices('GPU'))

1

# 读取数据

## SSH

In [3]:
data_ssh = nc.Dataset(r'D:/codeFile/jupyterDemo/3d_ts/BPNN/data/SSH/SSH_2005-2015_month.nc')
time_ssh = data_ssh['time'][:]
print('时间长度',len(time_ssh))

时间长度 132


In [4]:
# 确认经纬度，选取研究区域
ssh_lat = data_ssh['lat'][:].data   #0.125 -- 69.875
#print(ssh_lat[:])
ssh_lon = data_ssh['lon'][:].data   #110.125 -- 259.875
#print(ssh_lon[:])

In [5]:
ssh = data_ssh['ssh'][:].data
print(ssh.shape)
print(np.nanmax(ssh),np.nanmin(ssh))

(132, 280, 600)
2.20051 -0.43673334


## SSS

In [6]:
data_sss = nc.Dataset(r'D:/codeFile/jupyterDemo/3d_ts/BPNN/data/SSS/dataset-sss-ssd-rep-monthly_2005-2020.nc')

In [7]:
# 确定时间
time_sss = data_sss['time'][:132]
print(len(time_sss))

132


In [8]:
# 获取研究位置经纬度
sss_lat = data_sss['lat'][:].data   #0.125  -- 69.875 
#print(sss_lat[:])
sss_lon = data_sss['lon'][:].data   #110.125 -- 259.875
#print(sss_lon[:])

In [9]:
sss = data_sss['sos'][:132].data 
print(sss.shape)
sss = np.squeeze(sss)   # 移除大小为一的维度
print(sss.shape)

(132, 1, 280, 600)
(132, 280, 600)


In [10]:
print('最大值与最小值:',np.nanmax(sss),np.nanmin(sss))

最大值与最小值: 36.072178 13.604756


## SSW

In [11]:
data_ssw = nc.Dataset(r'D:/codeFile/jupyterDemo/3d_ts/BPNN/data/SSW/anon_ssw_2005_2018_Month_data.nc')
# 确定时间
time_ssw = data_ssw['time'][:132]
print(len(time_ssw))

132


In [12]:
lat_ssw = data_ssw['lat'][:].data   # lat 0.125    ----    69.875
#print(lat_ssw[314:594])
lon_ssw = data_ssw['lon'][:].data   # lon  110.125    ----    259.875
#print(lon_ssw[440:1040])

In [13]:
uwnd = data_ssw['uwnd'][:132,314:594,440:1040].data  
vwnd = data_ssw['vwnd'][:132,314:594,440:1040].data
print(uwnd.shape)
print(vwnd.shape)

(132, 280, 600)
(132, 280, 600)


In [14]:
np.nanmax(uwnd),np.nanmin(uwnd),np.nanmax(vwnd),np.nanmin(vwnd)

(12.433594, -11.7988205, 8.8359995, -13.645481)

## 3DT

In [15]:
data_3dt_1 = nc.Dataset(r'D:/codeFile/jupyterDemo/3d_ts/BPNN/data/3DT/dataset-armor-3d-rep-monthly_2005-2009.nc')
data_3dt_2 = nc.Dataset(r'D:/codeFile/jupyterDemo/3d_ts/BPNN/data/3DT/dataset-armor-3d-rep-monthly_2010-2015.nc')

In [16]:
sst_1 = data_3dt_1['to'][:].data
sst_2 = data_3dt_2['to'][:].data

In [17]:
print(sst_1.shape)
print(sst_2.shape)

(60, 43, 280, 600)
(72, 43, 280, 600)


In [18]:
temp_depth = np.append(sst_1,sst_2,axis=0)
print(temp_depth.shape)

(132, 43, 280, 600)


In [19]:
depths = [0,5,10,15,20,25,30,35,40,45,50,55,60,65,70,80,90,100,125,150,175,200,225,250,275,300,350,400,
         450,500,550,600,700,800,900,1000,1100,1200,1300,1400,1500,1750,2000]
#5-2000m
depths_use = [10,15,20,30,40,50,60,70,80,90,100,125,150,175,200,225,250,275,300,350,400,500,600,700,800,900,1000,1100,
              1300,1500,1750,2000]  # 长度为33

In [20]:
temp_depth_use = temp_depth[:,1:2]

In [21]:
for i,depth in enumerate(depths):
    if(depth in depths_use):
        temp_depth_use = np.concatenate((temp_depth_use,temp_depth[:,i:i+1]),axis=1)    

In [22]:
print(temp_depth_use.shape)
temp_depth_use = np.transpose(temp_depth_use,(0,2,3,1))
print(temp_depth_use.shape)

(132, 33, 280, 600)
(132, 280, 600, 33)


## 表面温度数据

In [23]:
sst = temp_depth[:,0:1,:,:]

In [24]:
sst = np.squeeze(sst)
print(sst.shape)

(132, 280, 600)


## 3DS

In [25]:
data_3ds = nc.Dataset(r'D:/codeFile/jupyterDemo/3d_ts/BPNN/data/3DT/sali/3DS_end_2005_2015.nc')
sali_depth = data_3ds['so'][:].data
print(sali_depth.shape)

(132, 43, 280, 600)


In [36]:
depths = [0,5,10,15,20,25,30,35,40,45,50,55,60,65,70,80,90,100,125,150,175,200,225,250,275,300,350,400,
         450,500,550,600,700,800,900,1000,1100,1200,1300,1400,1500,1750,2000]
#depths_use = [50,100,125,150,200,250,300,400,500,600,700,800,900,1000]
#5-2000m
depths_use = [10,15,20,30,40,50,60,70,80,90,100,125,150,175,200,225,250,275,300,350,400,500,600,700,800,900,1000,1100,
              1300,1500,1750,2000]  # 长度为33

In [37]:
sali_depth_use = sali_depth[:,1:2]

In [38]:
for i,depth in enumerate(depths):
    if(depth in depths_use):
        sali_depth_use = np.concatenate((sali_depth_use,sali_depth[:,i:i+1]),axis=1)    

In [39]:
print(sali_depth_use.shape)
sali_depth_use = np.transpose(sali_depth_use,(0,2,3,1))
print(sali_depth_use.shape)

(132, 33, 280, 600)
(132, 280, 600, 33)


## 数据合并

In [40]:
# 表面数据合并
print(ssh.shape,sst.shape,uwnd.shape,vwnd.shape,sss.shape)
cat_data =np.stack((ssh,sst,uwnd,vwnd,sss),axis=3)
cat_data.shape

(132, 280, 600) (132, 280, 600) (132, 280, 600) (132, 280, 600) (132, 280, 600)


(132, 280, 600, 5)

In [41]:
# #深度数据合并
# print(sali_depth_use.shape,temp_depth_use.shape)
# cat_depth = np.stack((sali_depth_use,temp_depth_use),axis=4)
# print(cat_depth.shape)
# cat_depth = np.reshape(cat_depth,(132, 280, 600,-1))
# print(cat_depth.shape)

In [42]:
# 将表面数据和水下数据在最后一个维度拼接，将空值删除。
cat_surface_depth = np.concatenate((cat_data,temp_depth_use),axis=-1)
print(cat_surface_depth.shape)

(132, 280, 600, 38)


## 将两个温度异常区域的数据设置为nan

In [43]:
# test = temp_depth_use[:,:,:,14:]
# test[test < 0.4] = np.nan
# test[test > 10] = np.nan
# cat_data_ss_depth[:,:,:,19:] = test
# cat_data_ss_depth.shape 

## 去掉合并数据中的nan


In [44]:
cat_surface_depth.shape

(132, 280, 600, 38)

In [45]:
## 先对合并的数据进行reshape成（长*宽，时间，通道）
cat_surface_depth_reshape  = np.transpose(cat_surface_depth,(1,2,0,3))
print(cat_surface_depth_reshape.shape)
cat_surface_depth_reshape = cat_surface_depth_reshape.reshape ((-1,132*38))
print(cat_surface_depth_reshape.shape)

(280, 600, 132, 38)
(168000, 5016)


In [46]:
# 将填充值赋值为nan
cat_surface_depth_reshape[cat_surface_depth_reshape == 32767] = np.nan
print('cat_surface_depth_reshape删除nan值之前',cat_surface_depth_reshape.shape)
# 使用 numpy.isnan 函数检测NaN值
nan_mask = np.isnan(cat_surface_depth_reshape)
# 通过布尔索引删除NaN值
cat_surface_depth_nonan = cat_surface_depth_reshape[~nan_mask.any(axis=1)]   
# ~nan_mask.any(axis=1)只要第一个维度中含有False就会返回False
print('cat_surface_depth_reshape删除nan值之后',cat_surface_depth_nonan.shape)

cat_surface_depth_reshape删除nan值之前 (168000, 5016)
cat_surface_depth_reshape删除nan值之后 (104729, 5016)


In [47]:
cat_surface_depth_nonan = cat_surface_depth_nonan.reshape(-1,132,38)
print(cat_surface_depth_nonan.shape)
cat_surface_depth_nonan = np.transpose(cat_surface_depth_nonan,(1,0,2))
print(cat_surface_depth_nonan.shape)

(104729, 132, 38)
(132, 104729, 38)


## 将数据进行堆叠

In [48]:
cat_surface_depth_nonan.shape
surface_data = cat_surface_depth_nonan[:,:,:5]
sali_depth_use = cat_surface_depth_nonan[:,:,5:]
print(surface_data.shape,sali_depth_use.shape)

(132, 104729, 5) (132, 104729, 33)


In [49]:
available = len(surface_data) 
predict = 1

surface_data_shape = surface_data.shape[1:]
sali_depth_use_shape = sali_depth_use.shape[1:]

N_stack = 6

X_len = available - (N_stack-1)
Y_len = X_len

X = np.zeros(shape = (X_len, N_stack,surface_data_shape[0],surface_data_shape[1]))  
for i in range(X_len):
    X[i] = np.stack(surface_data[i:i+N_stack])
    #print(i)

Y = np.zeros(shape = (Y_len,predict,sali_depth_use_shape[0],sali_depth_use_shape[1]))
for i in range(Y_len):
    Y[i] = np.stack(sali_depth_use[i+N_stack-1 : i+N_stack-1+predict])
   # print(i+N_stack-1)

print(X.shape)
print(Y.shape)

(127, 6, 104729, 5)
(127, 1, 104729, 33)


# 验证集和测试集的划分

In [50]:
# 最后十二个月的数据作为test
test_num = 115
x_train = X[:test_num]
y_train = Y[:test_num]

x_test = X[test_num:]
y_test = Y[test_num:]
print(x_train.shape,y_train.shape,x_test.shape,y_test.shape )

(115, 6, 104729, 5) (115, 1, 104729, 33) (12, 6, 104729, 5) (12, 1, 104729, 33)


## 修改维度

In [51]:
x_train = np.transpose(x_train, (0,2,1,3))   #(个数，尺度，时间，通道)
x_train = x_train.reshape((-1,6,5))          #(个数*尺度，时间，通道)

y_train = np.transpose(y_train, (0,2,1,3))
y_train = y_train.reshape((-1,1,33))

x_test = np.transpose(x_test, (0,2,1,3))
x_test = x_test.reshape((-1,6,5))

y_test = np.transpose(y_test, (0,2,1,3))
y_test = y_test.reshape((-1,1,33))
print(x_train.shape,y_train.shape,x_test.shape,y_test.shape)

(12043835, 6, 5) (12043835, 1, 33) (1256748, 6, 5) (1256748, 1, 33)


## 划分训练数据和目标数据

In [52]:
print('x_test.shape:',x_test.shape)
print('y_test.shape',y_test.shape)
print('x_train',x_train.shape)
print('y_train',y_train.shape)

x_test.shape: (1256748, 6, 5)
y_test.shape (1256748, 1, 33)
x_train (12043835, 6, 5)
y_train (12043835, 1, 33)


In [53]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.3, random_state=100)
print(x_train.shape,x_test.shape,x_val.shape,y_train.shape,y_test.shape,y_val.shape)

(8430684, 6, 5) (1256748, 6, 5) (3613151, 6, 5) (8430684, 1, 33) (1256748, 1, 33) (3613151, 1, 33)


# 特征归一化

In [54]:
# 记录数据集的形状
x_train_shape = x_train.shape
y_train_shape = y_train.shape

x_test_shape = x_test.shape
y_test_shape = y_test.shape

x_val_shape = x_val.shape
y_val_shape = y_val.shape

In [57]:
x_train.shape

(8430684, 6, 5)

In [58]:
scaler_f = StandardScaler()
x_train = scaler_f.fit_transform(x_train.reshape(-1,x_train_shape[2])) 
x_test = scaler_f.transform(x_test.reshape(-1,x_test_shape[2]))
x_val = scaler_f.transform(x_val.reshape(-1,x_val_shape[2]))

scaler_l = StandardScaler()
y_train = scaler_l.fit_transform(y_train.reshape(-1,1)) 
y_test = scaler_l.transform(y_test.reshape(-1,1)) 
y_val = scaler_l.transform(y_val.reshape(-1,1)) 

# scaler_l = StandardScaler()
# y_train = scaler_l.fit_transform(y_train.reshape(-1,y_train_shape[2])) 
# y_test = scaler_l.transform(y_test.reshape(-1,y_test_shape[2])) 
# y_val = scaler_l.transform(y_val.reshape(-1,y_val_shape[2])) 

In [59]:
x_train = np.reshape(x_train,(x_train_shape))
x_test = np.reshape(x_test,(x_test_shape))
x_val = np.reshape(x_val,(x_val_shape))
y_train = np.reshape(y_train,(y_train_shape[0],-1))
y_test = np.reshape(y_test,(y_test_shape[0],-1))
y_val = np.reshape(y_val,(y_val_shape[0],-1))

In [60]:
print(x_train.shape,x_test.shape,x_val.shape,y_train.shape,y_test.shape,y_val.shape)

(8430684, 6, 5) (1256748, 6, 5) (3613151, 6, 5) (8430684, 33) (1256748, 33) (3613151, 33)


# 定义网络结构

In [61]:
#  model2_input= Input(shape=(6,5))
#     x2 = MHA1(model2_input,model2_input)
#     x2 = LSTM(64, return_sequences=True)(x2)   
#     x2 = Dropout(0.2)(x2)
    
#     x2 = MHA2(x2,x2)
#     x2 = LSTM(32, return_sequences=True)(x2)
#     x2 = Dropout(0.2)(x2)
# #
#     x2 = MHA3(x2,x2)
#     x2 = Flatten()(x2)
#     out2 = Dense(1)(x2)

In [62]:
#  MHA2 = tf.keras.layers.MultiHeadAttention(num_heads=8, key_dim=8)
#     MHA3 = tf.keras.layers.MultiHeadAttention(num_heads=8, key_dim=4)
    
    
#     model2_input= Input(shape=(6,5))

#     x2 = LSTM(64, return_sequences=True)(model2_input)   
#     x2 = Dropout(0.2)(x2)
    
#     mha2 = MHA2(x2,x2)
#     x2 = tf.keras.layers.Add()([x2, mha2])
#     x2 = Activation('relu')(x2)
    
#     x2 = LSTM(32, return_sequences=True)(x2)
#     #x2 = Dropout(0.2)(x2)
# #
#     mha3 = MHA3(x2,x2)
#     x2 = tf.keras.layers.Add()([x2, mha3])
#     x2 = Activation('relu')(x2)
#     x2 = Flatten()(x2)
#     out2 = Dense(1)(x2)
    

In [63]:
# def my_model():
# #     MHA1 = tf.keras.layers.MultiHeadAttention(num_heads=2, key_dim=3)
#     MHA2 = tf.keras.layers.MultiHeadAttention(num_heads=4, key_dim=16)
# #     MHA3 = tf.keras.layers.MultiHeadAttention(num_heads=4, key_dim=8)
#     #MHA4 = tf.keras.layers.MultiHeadAttention(num_heads=8, key_dim=16)
    
#     model_input= Input(shape=(6,5))
    
# #     mha1 = MHA1(model_input,model_input)
# #     model = tf.keras.layers.Add()([model_input,mha1])
# #     model = tf.keras.layers.ReLU()(model)
    
#     model= LSTM(64, return_sequences=True)(model_input)   
#     model = Dropout(0.2)(model)
#     mha2 = MHA2(model,model)
#     model = tf.keras.layers.Add()([model,mha2])
#     model = tf.keras.layers.ReLU()(model)
    
#     model = LSTM(32, return_sequences=True)(model)
#     model = Dropout(0.2)(model)
# #     mha3 = MHA3(model,model)
# #     model = tf.keras.layers.Add()([model, mha3])
# #     model = tf.keras.layers.ReLU()(model)
    
# #     model_out = LSTM(32, return_sequences=True)(model)
# #     model = Dropout(0.3)(model)
# #     mha4 = MHA4(model,model)
# #     model = tf.keras.layers.Add()([model, mha4,model_out])
# #     model = tf.keras.layers.ReLU()(model)
    
    
#     model = Flatten()(model)
#     out = Dense(1)(model)
#     model = Model(inputs=model_input, outputs=out)
#     return model

In [64]:
# 注意力层的定制实现，以适应多次注意力调用
class Attention(Layer):
    def __init__(self, units=128, **kwargs):
        self.units = units
        super().__init__(**kwargs)
        
    def __call__(self, inputs):
        """
        Keras的多对一注意力机制。
        @param inputs: 3D tensor with shape (batch_size, time_steps, input_dim).
        @return: 2D tensor with shape (batch_size, 128)
        """
        
        hidden_states = inputs
        hidden_size = int(hidden_states.shape[2])  
        
        # Inside dense layer 内部全连接层
        #              hidden_states            dot               W            =>           score_first_part
        # (batch_size, time_steps, hidden_size) dot (hidden_size, hidden_size) => (batch_size, time_steps, hidden_size)
        # W是可训练权重矩阵 
        
        score_first_part = Dense(hidden_size, use_bias=False)(hidden_states) 
        # 进行一次全连接，本质上没有太大意义，只是为了增加一下模型复杂度，
        # 并且改变一下前T个时间步的特征表示,这里其实也可以不进行全连接，直接使用hidden_states
        
        #            score_first_part           dot        last_hidden_state     => attention_weights
        # (batch_size, time_steps, hidden_size) dot   (batch_size, hidden_size)  => (batch_size, time_steps)
        # # 注意力，增加对最后一个时间步长的关注
        
        h_t = Lambda(lambda x: x[:, -1, :], output_shape=(hidden_size,))(hidden_states)  # 取出最后一个时间步的特征
        score = Dot(axes=[1, 2])([h_t, score_first_part])  # 将最后一个时间步长与前T个时间步长进行点积
        attention_weights = Activation('softmax')(score)   # 求权重（将权值通过softmax进行归一化，得到直接可用的权重）
        
        
        # (batch_size, time_steps, hidden_size) dot (batch_size, time_steps) => (batch_size, hidden_size)
        context_vector = Dot(axes=[1, 1])([hidden_states, attention_weights])  # 上下文向量
        pre_activation = Concatenate()([context_vector, h_t]) #再次将最后一个时间步的信息加入，本质上是一个残差连接，这步可以省略
        attention_vector = Dense(self.units, use_bias=False, activation='tanh')(pre_activation) # 用一个全连接层进行合并
        
        print('注意力被执行了一次')
        return attention_vector

    def get_config(self):
        return {'units': self.units}

    @classmethod
    def from_config(cls, config):
        return cls(**config)

## 加入注意力机制

In [65]:
#@save
def masked_softmax(X, valid_lens):
    """通过在最后一个轴上掩蔽元素来执行softmax操作"""
    # X:3D张量，valid_lens:1D或2D张量
    if valid_lens is None:
        return tf.nn.softmax(X, axis=-1)
    else:
        shape = X.shape
        if len(valid_lens.shape) == 1:
            valid_lens = tf.repeat(valid_lens, repeats=shape[1])

        else:
            valid_lens = tf.reshape(valid_lens, shape=-1)
        # 最后一轴上被掩蔽的元素使用一个非常大的负值替换，从而其softmax输出为0
        X = d2l.sequence_mask(tf.reshape(X, shape=(-1, shape[-1])),
                              valid_lens, value=-1e6)
        return tf.nn.softmax(tf.reshape(X, shape=shape), axis=-1)

In [66]:
#@save   缩放点击注意力
class DotProductAttention(tf.keras.layers.Layer):
    """Scaleddotproductattention."""
    def __init__(self, dropout, **kwargs):
        super().__init__(**kwargs)
        self.dropout = tf.keras.layers.Dropout(dropout)

    # queries的形状：(batch_size，查询的个数，d)
    # keys的形状：(batch_size，“键－值”对的个数，d)
    # values的形状：(batch_size，“键－值”对的个数，值的维度)
    # valid_lens的形状:(batch_size，)或者(batch_size，查询的个数)
    def call(self, queries, keys, values, valid_lens, **kwargs):
        d = queries.shape[-1]
        scores = tf.matmul(queries, keys, transpose_b=True)/tf.math.sqrt(
            tf.cast(d, dtype=tf.float32))
        self.attention_weights = masked_softmax(scores, valid_lens)
        return tf.matmul(self.dropout(self.attention_weights, **kwargs), values)

In [67]:
#@save   多头注意力
class MultiHeadAttention_my(tf.keras.layers.Layer):
    """多头注意力"""
    def __init__(self, key_size, query_size, value_size, num_hiddens,
                 num_heads, dropout, bias=False, **kwargs):
        super().__init__(**kwargs)
        self.num_heads = num_heads
        self.key_size = key_size
        self.query_size = query_size
        self.value_size = value_size
        self.num_hiddens = num_hiddens
        self.dropout = dropout
        self.bias = bias
        self.attention = DotProductAttention(dropout)
        self.W_q = tf.keras.layers.Dense(num_hiddens, use_bias=bias)
        self.W_k = tf.keras.layers.Dense(num_hiddens, use_bias=bias)
        self.W_v = tf.keras.layers.Dense(num_hiddens, use_bias=bias)
        self.W_o = tf.keras.layers.Dense(num_hiddens, use_bias=bias)
        #print('初始化完成')

    def call(self, queries, keys, values, valid_lens, **kwargs):
        # queries，keys，values的形状:
        # (batch_size，查询或者“键－值”对的个数，num_hiddens)
        # valid_lens　的形状:
        # (batch_size，)或(batch_size，查询的个数)
        # 经过变换后，输出的queries，keys，values　的形状:
        # (batch_size*num_heads，查询或者“键－值”对的个数，num_hiddens/num_heads)
        
        #print(1)
        #print('queries.shape',queries.shape)
        #print('queries.type',type(queries))
        self.W_q(queries)
        #print(1.1)
        queries = transpose_qkv(self.W_q(queries), self.num_heads)
        keys = transpose_qkv(self.W_k(keys), self.num_heads)
        values = transpose_qkv(self.W_v(values), self.num_heads)
        #print(2)
        if valid_lens is not None:
            # 在轴0，将第一项（标量或者矢量）复制num_heads次，
            # 然后如此复制第二项，然后诸如此类。
            valid_lens = tf.repeat(valid_lens, repeats=self.num_heads, axis=0)
        #print(123)
        # output的形状:(batch_size*num_heads，查询的个数，num_hiddens/num_heads)
        #print('准备开始计算')
        output = self.attention(queries, keys, values, valid_lens, **kwargs)

        # output_concat的形状:(batch_size，查询的个数，num_hiddens)
        output_concat = transpose_output(output, self.num_heads)
        return self.W_o(output_concat)
    
    def get_config(self):
        return{'num_heads': self.num_heads,
                'key_size': self.key_size,
                'query_size': self.query_size,
                'value_size': self.value_size,
                'num_hiddens': self.num_hiddens,
                'dropout': self.dropout,
                'bias': self.bias}

In [68]:
#@save 形状转化
def transpose_qkv(X, num_heads):

    """为了多注意力头的并行计算而变换形状"""
    # 输入X的形状:(batch_size，查询或者“键－值”对的个数，num_hiddens)
    # 输出X的形状:(batch_size，查询或者“键－值”对的个数，num_heads，
    # num_hiddens/num_heads)
    #print('开始转化')
    #print(X.shape)
    #print(X.shape[2],X.shape[2]/num_heads)
    X = tf.reshape(X,shape=(-1, X.shape[1], num_heads, int(X.shape[2]/num_heads)))
    #print('转化完成')
    #print(X.shape)
    # 输出X的形状:(batch_size，num_heads，查询或者“键－值”对的个数,
    # num_hiddens/num_heads)
    X = tf.transpose(X, perm=(0, 2, 1, 3))

    # 最终输出的形状:(batch_size*num_heads,查询或者“键－值”对的个数,
    # num_hiddens/num_heads)
    return tf.reshape(X, shape=(-1, X.shape[2], X.shape[3]))


#@save
def transpose_output(X, num_heads):
    """逆转transpose_qkv函数的操作"""
    X = tf.reshape(X, shape=(-1, num_heads, X.shape[1], X.shape[2]))
    X = tf.transpose(X, perm=(0, 2, 1, 3))
    return tf.reshape(X, shape=(-1, X.shape[1],X.shape[2]*X.shape[3]))

In [71]:
def rest_back(model_input):
    model = LSTM(32, return_sequences=True)(model_input)
    model_attention = MultiHeadAttention_my(32, 32, 32, 32, 8, 0.5)(model, model, model, None)
    model = tf.keras.layers.Add()([model_attention, model])
    model = tf.keras.layers.ReLU()(model)
    return model 

def my_model():
    
    model_input= Input(shape=(6,5))
   
    #model= LSTM(64, return_sequences=True)(model_input)   
    #model = Dropout(0.2)(model)
    model = rest_back(model_input)
    model = rest_back(model)
    #model = rest_back(model)
    
    model = Flatten()(model)
    
    out = Dense(33)(model)
    model = Model(inputs=model_input, outputs=out)
    return model

In [72]:
# import graphviz
model = my_model()
model.summary()
# plot_model(model,show_shapes=True,show_layer_names=True)

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 6, 5)]       0                                            
__________________________________________________________________________________________________
lstm_3 (LSTM)                   (None, 6, 32)        4864        input_2[0][0]                    
__________________________________________________________________________________________________
multi_head_attention_my_3 (Mult (None, 6, 32)        4096        lstm_3[0][0]                     
                                                                 lstm_3[0][0]                     
                                                                 lstm_3[0][0]                     
____________________________________________________________________________________________

## 训练30 - 1000m

In [73]:
def train_model(x_train,y_train,x_val,y_val,name):
    # 创建网络
    model = my_model()
    # 编译网络
    model.compile(loss='mse', optimizer='adam')
    checkpoint_path=name
    keras_callbacks   = [
          EarlyStopping(monitor='val_loss', patience=16, mode='min', min_delta=0.0001),
          ModelCheckpoint(checkpoint_path, monitor='val_loss', save_best_only=True, mode='min')
    ]
    model.fit(x_train, y_train, validation_data=(x_val,y_val),
                    epochs=200, batch_size=10240, verbose=2, callbacks=keras_callbacks)
    return model

In [74]:
depth = [5,10,15,20,30,40,50,60,70,80,90,100,125,150,175,200,225,250,275,300,350,400,500,600,700,800,900,1000,1100,
              1300,1500,1750,2000] 

In [75]:
print(x_train.shape,x_test.shape,x_val.shape,y_train.shape,y_test.shape,y_val.shape)

(8430684, 6, 5) (1256748, 6, 5) (3613151, 6, 5) (8430684, 33) (1256748, 33) (3613151, 33)


In [76]:
# 一口气预测所有深度
model = train_model(x_train,y_train,x_val,y_val,'../model/LSTM/temp/LSTM_temp_(Mul)_all_3_m.h5')

Epoch 1/200
824/824 - 34s - loss: 0.0475 - val_loss: 0.0092
Epoch 2/200
824/824 - 29s - loss: 0.0081 - val_loss: 0.0056
Epoch 3/200
824/824 - 29s - loss: 0.0060 - val_loss: 0.0048
Epoch 4/200
824/824 - 29s - loss: 0.0052 - val_loss: 0.0044
Epoch 5/200
824/824 - 29s - loss: 0.0048 - val_loss: 0.0041
Epoch 6/200
824/824 - 29s - loss: 0.0045 - val_loss: 0.0040
Epoch 7/200
824/824 - 29s - loss: 0.0042 - val_loss: 0.0038
Epoch 8/200
824/824 - 29s - loss: 0.0041 - val_loss: 0.0037
Epoch 9/200
824/824 - 29s - loss: 0.0039 - val_loss: 0.0036
Epoch 10/200
824/824 - 29s - loss: 0.0038 - val_loss: 0.0035
Epoch 11/200
824/824 - 29s - loss: 0.0037 - val_loss: 0.0034
Epoch 12/200
824/824 - 29s - loss: 0.0036 - val_loss: 0.0034
Epoch 13/200
824/824 - 29s - loss: 0.0036 - val_loss: 0.0033
Epoch 14/200
824/824 - 29s - loss: 0.0035 - val_loss: 0.0033
Epoch 15/200
824/824 - 29s - loss: 0.0034 - val_loss: 0.0032
Epoch 16/200
824/824 - 29s - loss: 0.0034 - val_loss: 0.0032
Epoch 17/200
824/824 - 29s - loss

## 模型评估

In [77]:
# 评估函数定义
# 异常相关系数
import math
def acc(actual, predicted):
    pred_avg = np.average(predicted)

    act_avg = np.average(actual)
    diff_pred = predicted - pred_avg
    diff_act = actual - act_avg
    numerator = np.mean(np.sum(diff_pred*diff_act, axis=0))
    denominator = math.sqrt(np.mean(np.sum(diff_pred**2, axis=0)) * np.mean(np.sum(diff_act**2, axis=0)))
    ret_val = numerator/denominator
    return (100 * ret_val)


# 查看损失
def look_loss(history):
    plt.figure(figsize=(6.3, 2.5), dpi = 100)
    plt.plot(history.history['loss'], label='training data')
    plt.plot(history.history['val_loss'], label='validation data')
    plt.title('Loss')
    plt.ylabel('MSE Loss')
    plt.xlabel('Number of epochs')
    plt.grid()
    plt.legend(loc="upper right")
    plt.show()


# 模型预测
def all_estimate(best_model,x_test,y_test):
    testPred_1 = best_model.predict(x_test)
    a = y_test
    y_test_p = a.reshape(-1,1)
    testPred_p = testPred_1.reshape(-1,1)


    rmse = sqrt(mean_squared_error(y_test_p,testPred_p))
    print('Test RMSE: %.3f' % rmse)

    print('acc:',acc(y_test_p,testPred_p))

    r2 = r2_score(y_test_p,testPred_p)
    print("R² score:", r2)  
    

def respective_estimate(best_model,x_test,y_test):
    re2_list = []
    rmse_list = []
    for i in range(6):
        # 模型预测
        testPred = best_model.predict(x_test[i:i+1])
        a = y_test[i:i+1]
        y_test_p = a.reshape(-1,1)
        testPred_p = testPred.reshape(-1,1)
        r2 = r2_score(y_test_p,testPred_p)
        rmse = sqrt(mean_squared_error(y_test_p,testPred_p))
        re2_list.append(r2)
        rmse_list.append(rmse)    
    return re2_list,rmse_list

### 对指定层进行评估

In [78]:
best_model = tf.keras.models.load_model('D:/codeFile/jupyterDemo/3d_ts/BPNN/model/LSTM/temp/LSTM_temp_(Mul)_all_3_m.h5',custom_objects={'MultiHeadAttention_my': MultiHeadAttention_my})
all_estimate(best_model,x_test,y_test)

Test RMSE: 0.069
acc: 99.76896730373099
R² score: 0.9952103964652608


In [79]:
testPred = best_model.predict(x_test)

In [80]:
y_test.shape ,testPred.shape

((1256748, 33), (1256748, 33))

In [81]:
r2 = r2_score(y_test,testPred)
print("R² score:", r2)  

R² score: 0.9445564684762102


In [82]:
for i in range(0,33):
    print(depth[i],'m')
    rmse = sqrt(mean_squared_error(y_test[:,i:i+1],testPred[:,i:i+1]))
    print('Test RMSE: %.3f' % rmse)

    #print('acc:',acc(y_test[:,i:i+1],testPred[:,i:i+1]))

    r2 = r2_score(y_test[:,i:i+1],testPred[:,i:i+1])
    print("R² score:", r2)  

5 m
Test RMSE: 0.034
R² score: 0.9986220409183815
10 m
Test RMSE: 0.036
R² score: 0.998418288271928
15 m
Test RMSE: 0.038
R² score: 0.998206489989358
20 m
Test RMSE: 0.042
R² score: 0.9978211594200428
30 m
Test RMSE: 0.055
R² score: 0.9964325378341361
40 m
Test RMSE: 0.071
R² score: 0.9942691223872411
50 m
Test RMSE: 0.085
R² score: 0.9916162052392232
60 m
Test RMSE: 0.097
R² score: 0.9889334985785336
70 m
Test RMSE: 0.103
R² score: 0.9868969654975694
80 m
Test RMSE: 0.108
R² score: 0.9846564112248498
90 m
Test RMSE: 0.112
R² score: 0.9826194310220888
100 m
Test RMSE: 0.116
R² score: 0.9800898103921754
125 m
Test RMSE: 0.120
R² score: 0.9739974449675559
150 m
Test RMSE: 0.114
R² score: 0.9710953199779568
175 m
Test RMSE: 0.096
R² score: 0.9737798915843184
200 m
Test RMSE: 0.084
R² score: 0.9753385971616548
225 m
Test RMSE: 0.076
R² score: 0.9766613879574217
250 m
Test RMSE: 0.069
R² score: 0.9773647642685362
275 m
Test RMSE: 0.065
R² score: 0.9774829966910598
300 m
Test RMSE: 0.063
R² 

### 对所有层进行评估

In [308]:
# 模型预测
for i in range(0,33):
    best_model = tf.keras.models.load_model('D:/codeFile/jupyterDemo/3d_ts/BPNN/model/LSTM/sali/LSTM_sali_(Mul)_'+str(depth[i])+'m.h5',custom_objects={'MultiHeadAttention_my': MultiHeadAttention_my})
    print(str(depth[i])+'测试集：')
    all_estimate(best_model,x_test,y_test[:,i:i+1])
# re2_list,rmse_list = respective_estimate(best_model,x_test,y_test[:,:,:,:,0:5])
# re2_list,rmse_list

5测试集：
Test RMSE: 0.053
acc: 99.86382088075369
R² score: 0.9972548959436249
10测试集：
Test RMSE: 0.084
acc: 99.65136467907492
R² score: 0.992967099338938
15测试集：
Test RMSE: 0.088
acc: 99.6171009031483
R² score: 0.9923300461037283
20测试集：
Test RMSE: 0.093
acc: 99.57260468700282
R² score: 0.9914562140035097
30测试集：
Test RMSE: 0.108
acc: 99.41689524348295
R² score: 0.9883601689865469
40测试集：
Test RMSE: 0.123
acc: 99.25452732500521
R² score: 0.9849560603918971
50测试集：
Test RMSE: 0.138
acc: 99.05533723261198
R² score: 0.9811930629706401
60测试集：
Test RMSE: 0.149
acc: 98.91544409707845
R² score: 0.9784214494423065
70测试集：
Test RMSE: 0.157
acc: 98.82060411786448
R² score: 0.9764272294069934
80测试集：
Test RMSE: 0.158
acc: 98.79209415980196
R² score: 0.9759496981612152
90测试集：
Test RMSE: 0.168
acc: 98.629615832077
R² score: 0.9724825558827926
100测试集：
Test RMSE: 0.173
acc: 98.52393336264372
R² score: 0.9700996670908231
125测试集：
Test RMSE: 0.192
acc: 98.15124400417281
R² score: 0.9623612584690513
150测试集：
Test RM

OSError: SavedModel file does not exist at: D:/codeFile/jupyterDemo/3d_ts/BPNN/model/LSTM/sali/LSTM_sali_(Mul)_1750m.h5/{saved_model.pbtxt|saved_model.pb}

# 保存预测结果

In [ ]:
# 模型预测
result = [] 
for i in range(0,33):
    best_model = tf.keras.models.load_model('D:/codeFile/jupyterDemo/3d_ts/BPNN/model/LSTM/sali/LSTM_sali_'+str(depth[i])+'m.h5')
    #print('预测第'+str(depth[i]+'m的温度')
    testPred = best_model.predict(x_test)
    result.append(testPred)

In [ ]:
len(result),len(result[0]),len(result[0][0])

In [ ]:
result = np.array(result)
print(result.shape)
result = np.transpose(result,(1,2,0))
print(result.shape)
result = result.reshape(-1,33)
print(result.shape)

## 对预测结果进行反归一化

In [ ]:
result.shape,y_test.shape 

In [ ]:
result_unscaled = scaler_l.inverse_transform(result)
y_test_unscaled = scaler_l.inverse_transform(y_test)

## 评估反归一化之后的结果

In [ ]:
for i in range(0,33):
    r2 = r2_score(y_test_unscaled[:,i],result_unscaled[:,i])
    #print(str(depth[i])+'m R² score:',r2)
    print('%.4f' % r2)

In [ ]:
for i in range(0,33):
#     mse =  mean_squared_error(y_test_unscaled[:,i],result_unscaled[:,i])
#     print(str(depth[i])+'m Test mse: %.3f' % mse)
    rmse = sqrt(mean_squared_error(y_test_unscaled[:,i],result_unscaled[:,i]))
    print('%.3f' % rmse)

# 预测结果可视化

## 为归一化之后的预测值和真实值填充nan

In [ ]:
cat_surface_depth.shape 

In [ ]:
cat_surface_depth_test = np.transpose(cat_surface_depth,(1,2,0,3))
print(cat_surface_depth_test.shape)
cat_surface_depth_test = cat_surface_depth_test.reshape ((-1,132*38))
print(cat_surface_depth_test.shape)

In [ ]:
cat_surface_depth_test[cat_surface_depth_test == 32767] = np.nan
print('cat_surface_depth_test删除nan值之前',cat_surface_depth_test.shape)
# 使用 numpy.isnan 函数检测NaN值
nan_mask = np.isnan(cat_surface_depth_test)

In [ ]:
nan_mask[nan_mask.any(axis=1) == True] = True   # nan为True,只要有一个通道存在nan,就都设置为nan
nan_masks = nan_mask[:,1]    #得到图像的masks

In [ ]:
nan_masks.shape 

In [ ]:
### 创建一个形状与nan_masks相同且全是nan的数组
testPred_with_nan = np.full((168000,12*33), np.nan)  
print(testPred_with_nan.shape)

y_test_with_nan = np.full((168000,12*33), np.nan)  
print(y_test_with_nan.shape)

In [ ]:
result_unscaled_reshape = result_unscaled.reshape(12,-1,33)
print(result_unscaled_reshape.shape)
result_unscaled_reshape = np.transpose(result_unscaled_reshape,(1,0,2))
print(result_unscaled_reshape.shape)
result_unscaled_reshape = result_unscaled_reshape.reshape(-1,12*33)
print(result_unscaled_reshape.shape)

# ======================================== #
y_test_unscaled_reshape = y_test_unscaled.reshape(12,-1,33)
print(y_test_unscaled_reshape.shape)
y_test_unscaled_reshape = np.transpose(y_test_unscaled_reshape,(1,0,2))
print(y_test_unscaled_reshape.shape)
y_test_unscaled_reshape = y_test_unscaled_reshape.reshape(-1,12*33)
print(y_test_unscaled_reshape.shape)


In [ ]:
y_test_unscaled.shape 

In [ ]:
num = 0
for i,nan_mask in enumerate(nan_masks):   # i从0开始

    if (nan_mask == False): # False 代表没有nan，使用预测值进行填充
        testPred_with_nan[i] = result_unscaled_reshape[num]
        y_test_with_nan[i] = y_test_unscaled_reshape[num]
        num = num + 1
print(num)

In [ ]:
testPred_with_nan = testPred_with_nan.reshape(280, 600,12,33)
testPred_with_nan = np.transpose(testPred_with_nan,(2,3,0,1))


y_test_with_nan = y_test_with_nan.reshape(280, 600,12,33)
y_test_with_nan = np.transpose(y_test_with_nan,(2,3,0,1))

print(testPred_with_nan.shape,y_test_with_nan.shape)

# 结果可视化

In [ ]:
lat = []
lon = []
time = []

In [ ]:
lon.append(data_ssh['lon'][:].data)
lat.append(data_ssh['lat'][:].data)

In [ ]:
start_year = 2015
end_year =2015
time = []
date_range = pd.date_range(datetime(start_year,1,1),datetime(end_year+1,1,1),freq='1M')
len(date_range)
for i in range(len(date_range)):
    time.append(i)

In [ ]:
new_NC.close()

In [ ]:
new_NC = nc.Dataset("D:/codeFile/jupyterDemo/3d_ts/BPNN/model/LSTM/result/result5_2000m_sali_LSTM.nc", 'w', format='NETCDF4')

new_NC.createDimension('lat', len(lat[0]))
new_NC.createDimension('lon', len(lon[0]))
new_NC.createDimension('depth', 33)
new_NC.createDimension('time', len(time))

new_NC.createVariable('lat', 'f', ("lat"))
new_NC.createVariable('lon', 'f', ("lon"))
new_NC.createVariable('depth', 'f', ("depth"))
new_NC.createVariable('true_ss', 'f',("time","depth","lat","lon"))
new_NC.createVariable('pred_ss', 'f',("time","depth","lat","lon"))

time_var = new_NC.createVariable('time', 'f4',("time"))
time_var.units = 'months since 2015-01-15'
time_var.long_name = 'Months in Monthly Means'
time_var.axis = 'T'

#向变量中填充数据
new_NC.variables['lat'][:] = lat[0]
new_NC.variables['lon'][:] = lon[0]
new_NC.variables['time'][:] = np.array(time)
new_NC.variables['depth'][:] = depth

new_NC.variables['true_ss'][:]=np.array(y_test_with_nan)
new_NC.variables['pred_ss'][:]=np.array(testPred_with_nan)


#最后记得关闭文件
new_NC.close()
